<a href="https://colab.research.google.com/github/moonju815/moonju815/blob/main/convert_mp3toWav.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install flask pydub

In [2]:
from flask import Flask, request, send_from_directory, render_template_string
from pydub import AudioSegment
import os
from werkzeug.utils import secure_filename

app = Flask(__name__)

UPLOAD_FOLDER = 'uploads'
CONVERTED_FOLDER = 'converted'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(CONVERTED_FOLDER, exist_ok=True)

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['CONVERTED_FOLDER'] = CONVERTED_FOLDER

HTML_PAGE = '''
<!doctype html>
<title>MP3 to WAV 변환기</title>
<h1>MP3 파일 업로드해서 WAV로 변환</h1>
<form method=post enctype=multipart/form-data>
  <input type=file name=file accept=".mp3">
  <input type=submit value="변환">
</form>
{% if download_url %}
  <p>✅ 변환 완료! <a href="{{ download_url }}">여기서 다운로드</a></p>
{% endif %}
'''

@app.route('/', methods=['GET', 'POST'])
def upload_file():
    download_url = None
    if request.method == 'POST':
        file = request.files['file']
        if file.filename.endswith('.mp3'):
            filename = secure_filename(file.filename)
            mp3_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(mp3_path)

            wav_filename = filename.rsplit('.', 1)[0] + '.wav'
            wav_path = os.path.join(app.config['CONVERTED_FOLDER'], wav_filename)

            sound = AudioSegment.from_mp3(mp3_path)
            sound.export(wav_path, format='wav')

            download_url = f'/download/{wav_filename}'

    return render_template_string(HTML_PAGE, download_url=download_url)

@app.route('/download/<filename>')
def download_file(filename):
    return send_from_directory(app.config['CONVERTED_FOLDER'], filename, as_attachment=True)

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [6]:
!pip install flask pyngrok pydub

from flask import Flask, request, send_file, render_template_string
from pydub import AudioSegment
from pyngrok import ngrok
import os
from werkzeug.utils import secure_filename

app = Flask(__name__)

UPLOAD_FOLDER = '/content/uploads'
CONVERTED_FOLDER = '/content/converted'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(CONVERTED_FOLDER, exist_ok=True)

HTML_PAGE = '''
<!doctype html>
<title>MP3 to WAV 변환기</title>
<h1>MP3 파일 업로드해서 WAV로 변환</h1>
<form method=post enctype=multipart/form-data>
  <input type=file name=file accept=".mp3">
  <input type=submit value="변환">
</form>
{% if download_url %}
  <p>✅ 변환 완료! <a href="{{ download_url }}">여기서 다운로드</a></p>
{% endif %}
'''

@app.route('/', methods=['GET', 'POST'])
def upload_file():
    download_url = None
    if request.method == 'POST':
        file = request.files['file']
        if file and file.filename.endswith('.mp3'):
            filename = secure_filename(file.filename)
            mp3_path = os.path.join(UPLOAD_FOLDER, filename)
            file.save(mp3_path)

            wav_filename = filename.rsplit('.', 1)[0] + '.wav'
            wav_path = os.path.join(CONVERTED_FOLDER, wav_filename)

            sound = AudioSegment.from_mp3(mp3_path)
            sound.export(wav_path, format='wav')

            download_url = f'/download/{wav_filename}'

    return render_template_string(HTML_PAGE, download_url=download_url)

@app.route('/download/<filename>')
def download_file(filename):
    return send_file(os.path.join(CONVERTED_FOLDER, filename), as_attachment=True)

# 🔥 여기서 서버 실행
def start_ngrok():
    public_url = ngrok.connect(5000)
    print(f"🔗 외부 접속 링크: {public_url}")
    app.run(port=5000)

start_ngrok()


🔗 외부 접속 링크: NgrokTunnel: "https://571e-104-196-154-202.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [20/Apr/2025 08:16:46] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Apr/2025 08:16:47] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [20/Apr/2025 08:17:09] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Apr/2025 08:17:12] "GET /download/L1_Listen_and_Talk_A1.wav HTTP/1.1" 200 -


In [8]:
!pip install flask pyngrok pydub
import os
from flask import Flask, request, send_file, render_template_string
from werkzeug.utils import secure_filename
from pydub import AudioSegment
from pyngrok import ngrok
import zipfile

UPLOAD_FOLDER = '/content/uploads'
CONVERTED_FOLDER = '/content/converted'
ZIP_FILE = '/content/converted_wav_files.zip'

os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(CONVERTED_FOLDER, exist_ok=True)

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['CONVERTED_FOLDER'] = CONVERTED_FOLDER

HTML_PAGE = '''
<!doctype html>
<title>MP3 → WAV 일괄 변환기</title>
<h1>여러 MP3 파일 업로드해서 WAV로 일괄 변환</h1>
<form method=post enctype=multipart/form-data>
  <input type=file name=files multiple accept=".mp3">
  <input type=submit value="변환 시작">
</form>
{% if converted %}
  <h2>✅ 변환 완료!</h2>
  <ul>
    {% for file in converted %}
      <li><a href="/download/{{file}}">{{file}}</a></li>
    {% endfor %}
  </ul>
  <p><a href="/download_zip">📦 ZIP으로 모두 다운로드</a></p>
{% endif %}
'''

@app.route('/', methods=['GET', 'POST'])
def upload_files():
    converted = []
    if request.method == 'POST':
        files = request.files.getlist('files')
        for file in files:
            if file and file.filename.endswith('.mp3'):
                filename = secure_filename(file.filename)
                mp3_path = os.path.join(UPLOAD_FOLDER, filename)
                file.save(mp3_path)

                wav_filename = filename.rsplit('.', 1)[0] + '.wav'
                wav_path = os.path.join(CONVERTED_FOLDER, wav_filename)

                sound = AudioSegment.from_mp3(mp3_path)
                sound.export(wav_path, format='wav')
                converted.append(wav_filename)

        # ZIP으로 묶기
        with zipfile.ZipFile(ZIP_FILE, 'w') as zipf:
            for fname in converted:
                zipf.write(os.path.join(CONVERTED_FOLDER, fname), fname)

    return render_template_string(HTML_PAGE, converted=converted)

@app.route('/download/<filename>')
def download_file(filename):
    return send_file(os.path.join(CONVERTED_FOLDER, filename), as_attachment=True)

@app.route('/download_zip')
def download_zip():
    return send_file(ZIP_FILE, as_attachment=True)

# Ngrok으로 공개 링크 열기
from pyngrok import conf
conf.get_default().auth_token = "2vz7xgsxr8R7cjjhwBJ5aMnbSKn_6TtcyZ6mTocxtgmR2vFe2"

public_url = ngrok.connect(5000)
print(f"🔗 외부 접속 링크: {public_url}")
app.run(port=5000)


🔗 외부 접속 링크: NgrokTunnel: "https://df98-104-196-154-202.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [20/Apr/2025 08:20:25] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Apr/2025 08:20:25] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [20/Apr/2025 08:20:41] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Apr/2025 08:20:45] "GET /download_zip HTTP/1.1" 200 -


In [5]:
from pyngrok import conf, ngrok

# 여기에 너의 Ngrok 토큰을 붙여넣기!
conf.get_default().auth_token = "2vz7xgsxr8R7cjjhwBJ5aMnbSKn_6TtcyZ6mTocxtgmR2vFe2"
